# LSTM on real DNA sequences

# Leggi qui
Nella parte prima di START HERE ho commentato le linee che non servono più perché ho già generato i file necessari, che puoi trovare nella mia cartella /ubuntu (non dovrebbero esserci problemi di permessi). 

In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import subprocess
import numpy as np
from Bio import SeqIO

import tensorflow as tf

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD, Adam, Adadelta, RMSprop
from keras.layers import Conv1D, Dense, MaxPooling1D, Flatten, Dropout
from keras.layers import Embedding, GlobalAveragePooling1D, LSTM, SimpleRNN, GRU

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import itertools

Using TensorFlow backend.


In [2]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
intr_coords = pd.read_csv('Akey_intr_coords.bed', header = None, sep = "\t")
intr_coords.head()

,0,1,2
0,chr1,2903159,2915884
1,chr1,2932446,2972497
2,chr1,2960608,2996556
3,chr1,2960608,2999518
4,chr1,2960608,3001253


In [4]:
intr_lengths = intr_coords.iloc[:, 2]-intr_coords.iloc[:, 1]

In [5]:
print(stats.describe(intr_lengths))

DescribeResult(nobs=83601, minmax=(10002, 1194940), mean=92137.5803877944, variance=6139175379.516866, skewness=2.7095069302166377, kurtosis=11.38558261567325)


In [6]:
#!zcat hg19.fa.gz | bgzip -c > hg19.fa.bgz

In [16]:
a = 0
with open('hg19_intr_regions.fa', 'a') as fp:
    for i in range(intr_coords.shape[0]):
        coord = str(str(intr_coords.iloc[i, 0]) + ':' 
                    + str(intr_coords.iloc[i, 1]) + '-' + str(intr_coords.iloc[i, 2]))
        subprocess.run(['samtools', 'faidx', 'hg19.fa.bgz', str(coord)], stdout = fp)
        a = a + 1
        if a%10000 == 0:
            print('Finished ' + str(a) + ' Neanderthal introgressed haplotypes')

Finished 10000 Neanderthal introgressed haplotypes
Finished 20000 Neanderthal introgressed haplotypes
Finished 30000 Neanderthal introgressed haplotypes
Finished 40000 Neanderthal introgressed haplotypes
Finished 50000 Neanderthal introgressed haplotypes
Finished 60000 Neanderthal introgressed haplotypes
Finished 70000 Neanderthal introgressed haplotypes
Finished 80000 Neanderthal introgressed haplotypes


In [8]:
"""chr_sizes = pd.read_csv("hg19.fa.bgz.fai", header = None, sep = "\t")
chr_sizes = chr_sizes.drop([2, 3, 4], axis = 1)
chr_sizes.head()"""

'chr_sizes = pd.read_csv("hg19.fa.bgz.fai", header = None, sep = "\t")\nchr_sizes = chr_sizes.drop([2, 3, 4], axis = 1)\nchr_sizes.head()'

In [9]:
"""chr_list = []
start_list = []
end_list = []
intr_lengths = list(intr_coords.iloc[:, 2] - intr_coords.iloc[:, 1])
a = 0
for i in range(intr_coords.shape[0]):
    chr_df = intr_coords[intr_coords[0].isin([intr_coords.iloc[i,0]])]
    overlap = True
    while overlap == True:
        reg_start = np.random.randint(1, int(chr_sizes[chr_sizes[0] == intr_coords.iloc[i,0]].iloc[:,1]))
        reg_end = reg_start + intr_lengths[i]
        for j in range(chr_df.shape[0]):
            b1 = chr_df.iloc[j,1]
            b2 = chr_df.iloc[j,2]
            if (reg_start > b1 and reg_start < b2) or (reg_end > b1 and reg_end < b2) or \
            (b1 > reg_start and b1 < reg_end) or (b2 > reg_start and b2 < reg_end):
                overlap = True
                break
            else:
                overlap = False
    chr_list.append(intr_coords.iloc[i,0])
    start_list.append(reg_start)
    end_list.append(reg_end)
    a = a + 1
    if a%10000 == 0:
            print('Finished ' + str(a) + ' Neanderthal introgressed haplotypes')
depl_coords = pd.DataFrame({'0': chr_list, '1': start_list, '2': end_list})
depl_coords.to_csv("Akey_depl_coords.bed", index = False, header = False, sep = "\t")
depl_coords.head()"""

'chr_list = []\nstart_list = []\nend_list = []\nintr_lengths = list(intr_coords.iloc[:, 2] - intr_coords.iloc[:, 1])\na = 0\nfor i in range(intr_coords.shape[0]):\n    chr_df = intr_coords[intr_coords[0].isin([intr_coords.iloc[i,0]])]\n    overlap = True\n    while overlap == True:\n        reg_start = np.random.randint(1, int(chr_sizes[chr_sizes[0] == intr_coords.iloc[i,0]].iloc[:,1]))\n        reg_end = reg_start + intr_lengths[i]\n        for j in range(chr_df.shape[0]):\n            b1 = chr_df.iloc[j,1]\n            b2 = chr_df.iloc[j,2]\n            if (reg_start > b1 and reg_start < b2) or (reg_end > b1 and reg_end < b2) or             (b1 > reg_start and b1 < reg_end) or (b2 > reg_start and b2 < reg_end):\n                overlap = True\n                break\n            else:\n                overlap = False\n    chr_list.append(intr_coords.iloc[i,0])\n    start_list.append(reg_start)\n    end_list.append(reg_end)\n    a = a + 1\n    if a%10000 == 0:\n            print(\'Fini

In [10]:
depl_coords = pd.read_csv("Akey_depl_coords.bed", sep = "\t")

In [11]:
!bedtools intersect -a Akey_intr_coords.bed -b Akey_depl_coords.bed | wc -l

0


In [12]:
a = 0
with open('hg19_depl_regions.fa', 'a') as fp:
    for i in range(depl_coords.shape[0]):
        coord = str(str(depl_coords.iloc[i, 0]) + ':' 
                    + str(depl_coords.iloc[i, 1]) + '-' + str(depl_coords.iloc[i, 2]))
        subprocess.run(['samtools', 'faidx', 'hg19.fa.bgz', str(coord)], stdout = fp)
        a = a + 1
        if a%10000 == 0:
            print('Finished ' + str(a) + ' Neanderthal ancestry depleted regions')

Finished 10000 Neanderthal ancestry depleted regions
Finished 20000 Neanderthal ancestry depleted regions
Finished 30000 Neanderthal ancestry depleted regions
Finished 40000 Neanderthal ancestry depleted regions
Finished 50000 Neanderthal ancestry depleted regions
Finished 60000 Neanderthal ancestry depleted regions
Finished 70000 Neanderthal ancestry depleted regions
Finished 80000 Neanderthal ancestry depleted regions


In [13]:
!grep -c N hg19_intr_regions.fa

grep: hg19_intr_regions.fa: No such file or directory


In [14]:
!grep -c N hg19_depl_regions.fa

18502152


In [17]:
intr_file = 'hg19_intr_regions.fa'
depl_file = 'hg19_depl_regions.fa'
a = 0
i = 0
with open('hg19_intr_clean.fa', 'a') as intr_out, open('hg19_depl_clean.fa', 'a') as depl_out:
    for intr, depl in zip(SeqIO.parse(intr_file, 'fasta'), SeqIO.parse(depl_file, 'fasta')):
        upper_intr = intr.seq.upper()
        upper_depl = depl.seq.upper()
        a = a + 1
        if a%10000 == 0:
            print('Finished ' + str(a) + ' entries')
        if 'N' not in str(upper_intr) and 'N' not in str(upper_depl):
            intr.seq = upper_intr
            SeqIO.write(intr, intr_out, 'fasta')
            depl.seq = upper_depl
            SeqIO.write(depl, depl_out, 'fasta')
            i = i + 1
        else:
            continue
print('We have processed ' + str(a) + ' entries and written ' + str(i) + ' entries to two fasta-files')

Finished 10000 entries
Finished 20000 entries
Finished 30000 entries
Finished 40000 entries
Finished 50000 entries
Finished 60000 entries
Finished 70000 entries
Finished 80000 entries
We have processed 83601 entries and written 73364 entries to two fasta-files


In [18]:
!grep -c N hg19_intr_regions.fa

863


In [19]:
!grep -c N hg19_depl_regions.fa

18502152


# START HERE

Qui il codice commentato è perché volevo fare un po' di prove, quindi può essere cambiato a piacimento

Attenzione perché la rete così com'è ci mette tanto ad allenarsi.

In [38]:
intr_file = 'hg19_intr_clean.fa'
depl_file = 'hg19_depl_clean.fa'

"""a = 0
intr_seqs = []
depl_seqs = []
for intr, depl in zip(SeqIO.parse(intr_file, 'fasta'), SeqIO.parse(depl_file, 'fasta')):
    cut = 1000
    if len(str(intr.seq)) < cut or len(str(depl.seq)) < cut:
        continue
    s_intr = str(intr.seq)[0:cut]
    s_depl = str(depl.seq)[0:cut]
    if s_intr.count('A')>0 and s_intr.count('C')>0 and s_intr.count('G')>0 and s_intr.count('T')>0 and \
    s_depl.count('A')>0 and s_depl.count('C')>0 and s_depl.count('G')>0 and s_depl.count('T')>0:
        intr_seqs.append(s_intr)
        depl_seqs.append(s_depl)
    a = a + 1
    if a%10000 == 0:
        print('Finished ' + str(a) + ' entries')"""

e = 0
intr_seqs = []
depl_seqs = []
for intr, depl in zip(SeqIO.parse(intr_file, 'fasta'), SeqIO.parse(depl_file, 'fasta')):
    
    #cutoff = 200
    #my_intr_seq = str(intr.seq)[0:cutoff]
    #my_depl_seq = str(depl.seq)[0:cutoff]
    #intr_seqs.append(my_intr_seq)
    #depl_seqs.append(my_depl_seq)
    
    step = 200; jump = 1; a = 0; b = step; n_jumps = 5
    for j in range(n_jumps):
        s_intr = str(intr.seq)[a:b]
        s_depl = str(depl.seq)[a:b]
        intr_seqs.append(s_intr)
        depl_seqs.append(s_depl)
        a = a + jump
        b = a + step
    
    e = e + 1
    if e%20000 == 0:
        print('Finished ' + str(e) + ' entries')
        
def getKmers(sequence, size):
    return [sequence[x:x+size].upper() for x in range(len(sequence) - size + 1)]

kmer = 10
intr_texts = [' '.join(getKmers(i, kmer)) for i in intr_seqs]
depl_texts = [' '.join(getKmers(i, kmer)) for i in depl_seqs]

Finished 20000 entries
Finished 40000 entries
Finished 60000 entries


In [39]:
"""sequences = intr_seqs + depl_seqs
len(sequences)"""

merge_texts = intr_texts + depl_texts
len(merge_texts)

733640

In [40]:
"""labels = list(np.ones(len(intr_seqs))) + list(np.zeros(len(depl_seqs)))
len(labels)"""

labels = list(np.ones(len(intr_texts))) + list(np.zeros(len(depl_texts)))
print(len(labels))

733640


In [41]:
"""e = 0
intr_seqs = []
depl_seqs = []
for intr, depl in zip(SeqIO.parse(intr_file, 'fasta'), SeqIO.parse(depl_file, 'fasta')):
    
    cutoff = 200
    my_intr_seq = str(intr.seq)[0:cutoff]
    my_depl_seq = str(depl.seq)[0:cutoff]
    
    intr_seqs.append(my_intr_seq)
    depl_seqs.append(my_depl_seq)
    
    e = e + 1
    if e%20000 == 0:
        print('Finished ' + str(e) + ' entries')
        
def getKmers(sequence, size):
    return [sequence[x:x+size].upper() for x in range(len(sequence) - size + 1)]

kmer = 10
intr_texts = [' '.join(getKmers(i, kmer)) for i in intr_seqs]
depl_texts = [' '.join(getKmers(i, kmer)) for i in depl_seqs]"""

"e = 0\nintr_seqs = []\ndepl_seqs = []\nfor intr, depl in zip(SeqIO.parse(intr_file, 'fasta'), SeqIO.parse(depl_file, 'fasta')):\n    \n    cutoff = 200\n    my_intr_seq = str(intr.seq)[0:cutoff]\n    my_depl_seq = str(depl.seq)[0:cutoff]\n    \n    intr_seqs.append(my_intr_seq)\n    depl_seqs.append(my_depl_seq)\n    \n    e = e + 1\n    if e%20000 == 0:\n        print('Finished ' + str(e) + ' entries')\n        \ndef getKmers(sequence, size):\n    return [sequence[x:x+size].upper() for x in range(len(sequence) - size + 1)]\n\nkmer = 10\nintr_texts = [' '.join(getKmers(i, kmer)) for i in intr_seqs]\ndepl_texts = [' '.join(getKmers(i, kmer)) for i in depl_seqs]"

In [42]:
"""merge_texts = intr_texts + depl_texts
len(merge_texts)"""

'merge_texts = intr_texts + depl_texts\nlen(merge_texts)'

In [43]:
"""labels = list(np.ones(len(intr_texts))) + list(np.zeros(len(depl_texts)))
print(len(labels))"""

'labels = list(np.ones(len(intr_texts))) + list(np.zeros(len(depl_texts)))\nprint(len(labels))'

In [44]:
#cv = CountVectorizer()
#X = cv.fit_transform(merge_texts)

#tfidf_transformer = TfidfTransformer()
#X = tfidf_transformer.fit_transform(X)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(merge_texts)
#X = tokenizer.texts_to_matrix(merge_texts, mode = 'freq')

encoded_docs = tokenizer.texts_to_sequences(merge_texts)
max_length = max([len(s.split()) for s in merge_texts])
X = pad_sequences(encoded_docs, maxlen = max_length, padding = 'post')

print(X)
print('\n')
print(X.shape)

[[153634 568397 382160 ... 635443 506832 658088]
 [568397 382160 380712 ... 506832 658088 654350]
 [382160 380712 323461 ... 658088 654350 503434]
 ...
 [  1173    900    397 ...  64841  93299 115195]
 [   900    397    537 ...  93299 115195 135311]
 [   397    537   1117 ... 115195 135311  63747]]


(733640, 191)


In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size = 0.20, random_state = 42)

In [46]:
print(X_train.shape)
print(X_test.shape)

(586912, 191)
(146728, 191)


In [62]:
X_train1 = X_train[:200000]
X_test1 = X_test[:5000]
y_train1 = y_train[:200000]
y_test1 = y_test[:5000]
print(X_train.shape)
print(X_test.shape)

(586912, 191)
(146728, 191)


In [63]:
max_length = max([len(s.split()) for s in merge_texts])
print(max_length)

191


In [64]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

944238


In [65]:
"""model = Sequential()
#model.add(Dense(3000, input_shape = (X.shape[1],), activation = 'sigmoid'))
model.add(Embedding(vocab_size, 32, input_length = max_length, dropout = 0.2))
#model.add(Conv1D(filters = 16, kernel_size = 5, padding = 'same', activation = 'relu'))
#model.add(MaxPooling1D(pool_size = 2))
#model.add(LSTM(10)) #dropout = 0.2, recurrent_dropout = 0.2
#model.add(GRU(10))
model.add(SimpleRNN(10, dropout = 0.2, recurrent_dropout = 0.2))
#model.add(Flatten())
#model.add(Dense(10, activation = 'sigmoid'))
model.add(Dense(1, activation = 'sigmoid'))

epochs = 5
model.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])
#model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#model.compile(loss = 'binary_crossentropy', optimizer = 'SGD', metrics = ['accuracy'])
#model.compile(loss = 'binary_crossentropy', optimizer = RMSprop(lr = 0.0001), metrics = ['accuracy'])
checkpoint = ModelCheckpoint("weights.best.hdf5", monitor = 'val_acc', verbose = 1, 
                             save_best_only = True, mode = 'max')
print(model.summary())"""

'model = Sequential()\n#model.add(Dense(3000, input_shape = (X.shape[1],), activation = \'sigmoid\'))\nmodel.add(Embedding(vocab_size, 32, input_length = max_length, dropout = 0.2))\n#model.add(Conv1D(filters = 16, kernel_size = 5, padding = \'same\', activation = \'relu\'))\n#model.add(MaxPooling1D(pool_size = 2))\n#model.add(LSTM(10)) #dropout = 0.2, recurrent_dropout = 0.2\n#model.add(GRU(10))\nmodel.add(SimpleRNN(10, dropout = 0.2, recurrent_dropout = 0.2))\n#model.add(Flatten())\n#model.add(Dense(10, activation = \'sigmoid\'))\nmodel.add(Dense(1, activation = \'sigmoid\'))\n\nepochs = 5\nmodel.compile(loss = \'binary_crossentropy\', optimizer = \'rmsprop\', metrics = [\'accuracy\'])\n#model.compile(loss = \'binary_crossentropy\', optimizer = \'adam\', metrics = [\'accuracy\'])\n#model.compile(loss = \'binary_crossentropy\', optimizer = \'SGD\', metrics = [\'accuracy\'])\n#model.compile(loss = \'binary_crossentropy\', optimizer = RMSprop(lr = 0.0001), metrics = [\'accuracy\'])\nche

In [66]:
from keras.layers import Embedding, GlobalAveragePooling1D, LSTM, SimpleRNN, GRU, Bidirectional

model = Sequential()
model.add(Embedding(vocab_size, 10)) #dropout = 0.2 #input_length = max_length
#model.add(Conv1D(filters = 16, kernel_size = 5, padding = 'same', activation = 'relu'))
#model.add(MaxPooling1D(pool_size = 2))
#model.add(LSTM(10)) #dropout = 0.2, recurrent_dropout = 0.2
model.add(Bidirectional(LSTM(10))) #dropout = 0.2, recurrent_dropout = 0.2
#model.add(Bidirectional(SimpleRNN(10)))
#model.add(GRU(10))
#model.add(SimpleRNN(10, dropout = 0.2, recurrent_dropout = 0.2))
#model.add(Flatten())
model.add(Dense(10, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

epochs = 5
model.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])
#model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#model.compile(loss = 'binary_crossentropy', optimizer = 'SGD', metrics = ['accuracy'])
#model.compile(loss = 'binary_crossentropy', optimizer = RMSprop(lr = 0.0001), metrics = ['accuracy'])
checkpoint = ModelCheckpoint("weights.best.hdf5", monitor = 'val_acc', verbose = 1, 
                             save_best_only = True, mode = 'max')
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, None, 10)          9442380   
_________________________________________________________________
bidirectional_7 (Bidirection (None, 20)                1680      
_________________________________________________________________
dense_13 (Dense)             (None, 10)                210       
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 11        
Total params: 9,444,281
Trainable params: 9,444,281
Non-trainable params: 0
_________________________________________________________________
None


In [67]:
history = model.fit(X_train1, y_train1, 
                    epochs = epochs, verbose = 1, validation_split = 0.2, batch_size = 32, shuffle = True, 
                    callbacks = [checkpoint])

Train on 160000 samples, validate on 40000 samples
Epoch 1/5
   576/160000 [..............................] - ETA: 35:13 - loss: 0.6925 - accuracy: 0.5278

KeyboardInterrupt: 

In [ ]:
"""plt.figure()

predicted_labels = model.predict(X_test)
cm = confusion_matrix(y_test, [np.round(i[0]) for i in predicted_labels])

cm = cm.astype('float') / cm.sum(axis = 1)[:, np.newaxis]

plt.imshow(cm, cmap = plt.cm.Blues)
plt.title('Normalized confusion matrix', fontsize = 15)
plt.colorbar()
plt.xlabel('True label', fontsize = 15)
plt.ylabel('Predicted label', fontsize = 15)
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm[i, j], '.2f'),
             horizontalalignment = 'center', verticalalignment = 'center', fontsize = 20,
             color='white' if cm[i, j] > 0.5 else 'black')
plt.show()"""

plt.figure(figsize = (15,10))

predicted_labels = model.predict(X_test1)
cm = confusion_matrix(y_test1, [np.round(i[0]) for i in predicted_labels])
print('Confusion matrix:\n',cm)

cm = cm.astype('float') / cm.sum(axis = 1)[:, np.newaxis]

plt.imshow(cm, cmap = plt.cm.Blues)
plt.title('Normalized confusion matrix', fontsize = 20)
plt.colorbar()
plt.xlabel('True label', fontsize = 20)
plt.ylabel('Predicted label', fontsize = 20)
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm[i, j], '.2f'),
             horizontalalignment = 'center', verticalalignment = 'center', fontsize = 20,
             color='white' if cm[i, j] > 0.5 else 'black')
plt.show()

In [ ]:
scores = model.evaluate(X_test1, y_test1, verbose = 0)
print("Accuracy: %.2f%%" % (scores[1]*100))